In [16]:
import csv
import os
import cv2
import glob
import math
import time
import random
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
PIXELS = 100
imageSize = PIXELS * PIXELS
num_features = imageSize 

def load_train_cv(encoder):
    X_train = []
    y_train = []
    print('Read train images')
    with open('id_train.csv', 'rt') as csvfile:
        trainreader = csv.reader(csvfile, delimiter=',')
        next(trainreader)
        for row in trainreader:
            file_name = os.path.join('input', row[0] + '.jpg')
            img = cv2.imread(file_name,0)
            img_width = img.shape[0]
            img_height = img.shape[1]
            img = cv2.resize(img, (PIXELS, PIXELS))
            feature = []
            feature.append(img_width)
            feature.append(img_height)
            feature = feature + np.sum(img, 0).tolist()
            feature = feature + np.sum(img, 1).tolist()
            feature = feature + np.var(img, 0).tolist()
            feature = feature + np.var(img, 1).tolist()
            
            X_train.append(feature)
            y_train.append(row[1])
            
            M = cv2.getRotationMatrix2D((50,50),90,1)
            
            img90 = cv2.warpAffine(img, M, (100, 100))
            feature = []
            feature.append(img_height)
            feature.append(img_width)
            feature = feature + np.sum(img90, 0).tolist()
            feature = feature + np.sum(img90, 1).tolist()
            feature = feature + np.var(img90, 0).tolist()
            feature = feature + np.var(img90, 1).tolist()

            X_train.append(feature)
            
            if row[1] == '1':
                y_train.append ('2')
            elif row[1] == '2':
                y_train.append ('1')
            else:
                y_train.append(row[1])
            img180 = cv2.warpAffine(img90, M, (100, 100))
            
            feature = []
            feature.append(img_height)
            feature.append(img_width)
            feature = feature + np.sum(img180, 0).tolist()
            feature = feature + np.sum(img180, 1).tolist()
            feature = feature + np.var(img180, 0).tolist()
            feature = feature + np.var(img180, 1).tolist()

            X_train.append(feature)
            
            y_train.append(row[1])
            
            img270 = cv2.warpAffine(img180, M, (100, 100))
            
            feature = []
            feature.append(img_height)
            feature.append(img_width)
            feature = feature + np.sum(img270, 0).tolist()
            feature = feature + np.sum(img270, 1).tolist()
            feature = feature + np.var(img270, 0).tolist()
            feature = feature + np.var(img270, 1).tolist()

            X_train.append(feature)
            
            if row[1] == '1':
                y_train.append ('2')
            elif row[1] == '2':
                y_train.append ('1')
            else:
                y_train.append(row[1])

    X_train = np.array(X_train)
    y_train = np.array(y_train)#.astype('int32')

    y_train = encoder.fit_transform(y_train).astype('int32')

    X_train, y_train = shuffle(X_train, y_train)

    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2)

    #X_train = X_train.reshape(X_train.shape[0], 1, PIXELS, PIXELS).astype('float32') / 255.
    #X_test = X_test.reshape(X_test.shape[0], 1, PIXELS, PIXELS).astype('float32') / 255.

    return X_train, y_train, X_test, y_test, encoder

In [27]:
def load_test():
    print('Read test images')
    X_test = []
    X_test_id = []
    with open('sample_submission4.csv', 'rt') as csvfile:
        testreader = csv.reader(csvfile, delimiter=',')
        next(testreader)
        for row in testreader:
            file_name = os.path.join('input', row[0] + '.jpg')
            img = cv2.imread(file_name,0)
            img_width = img.shape[0]
            img_height = img.shape[1]
            img = cv2.resize(img, (PIXELS, PIXELS))
            feature = []
            feature.append(img_width)
            feature.append(img_height)
            feature = feature + np.sum(img, 0).tolist()
            feature = feature + np.sum(img, 1).tolist()
            feature = feature + np.var(img, 0).tolist()
            feature = feature + np.var(img, 1).tolist()
            
            X_test.append(feature)
            X_test_id.append(row[0])

    X_test = np.array(X_test)
    X_test_id = np.array(X_test_id)
    return X_test, X_test_id

# load data
X_test, X_test_id = load_test()
print('Test shape:', X_test.shape, 'Test ID shape:', X_test_id.shape)

Read test images
('Test shape:', (13999L, 402L), 'Test ID shape:', (13999L,))


In [29]:
train_y[0:10]

array([0, 0, 3, 2, 1, 1, 3, 0, 0, 2])

In [13]:
'''
load training data and test data 
'''
encoder = LabelEncoder()

# load the training and validation data sets
train_X, train_y, valid_X, valid_y, encoder = load_train_cv(encoder)
print('Train shape:', train_X.shape, 'Dev (valid) shape:', valid_X.shape)
print('Train shape:', train_y.shape, 'Dev (valid) shape:', valid_y.shape)

Read train images
('Train shape:', (25600L, 402L), 'Dev (valid) shape:', (6400L, 402L))
('Train shape:', (25600L,), 'Dev (valid) shape:', (6400L,))
Read test images
('Test shape:', (13999L, 100L, 100L), 'Test ID shape:', (13999L,))


In [14]:
#Train RF
RF = RandomForestClassifier(n_estimators = 100)
RF.fit(train_X, train_y)
print(RF.score(valid_X, valid_y))

0.59703125


In [18]:
#Train GBM with initial settings
gbm0 = GradientBoostingClassifier(random_state=10)
gbm0.fit(train_X, train_y)
print(gbm0.score(valid_X, valid_y))

0.61453125


In [20]:
#Let's tune for n_estimators
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, \
                                                               min_samples_split=500,\
                                                               min_samples_leaf=50,\
                                                               max_depth=8,max_features='sqrt',\
                                                               subsample=0.8,\
                                                               random_state=10), 
param_grid = param_test1,n_jobs=4,iid=False, cv=5)
gsearch1.fit(train_X,train_y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.59297, std: 0.00877, params: {'n_estimators': 20},
  mean: 0.61102, std: 0.00645, params: {'n_estimators': 30},
  mean: 0.62379, std: 0.00645, params: {'n_estimators': 40},
  mean: 0.63309, std: 0.00663, params: {'n_estimators': 50},
  mean: 0.63957, std: 0.00503, params: {'n_estimators': 60},
  mean: 0.64457, std: 0.00422, params: {'n_estimators': 70},
  mean: 0.64828, std: 0.00264, params: {'n_estimators': 80}],
 {'n_estimators': 80},
 0.648281157574221)

In [21]:
# Pick n_estimators=80
#Lets tune for max_depth and min_samples_split
param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=80, \
                                                               min_samples_leaf=50,\
                                                               max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2,n_jobs=4,iid=False, cv=5)
gsearch2.fit(train_X,train_y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: 0.63316, std: 0.00305, params: {'min_samples_split': 200, 'max_depth': 5},
  mean: 0.63027, std: 0.00252, params: {'min_samples_split': 400, 'max_depth': 5},
  mean: 0.63133, std: 0.00345, params: {'min_samples_split': 600, 'max_depth': 5},
  mean: 0.62547, std: 0.00339, params: {'min_samples_split': 800, 'max_depth': 5},
  mean: 0.62406, std: 0.00384, params: {'min_samples_split': 1000, 'max_depth': 5},
  mean: 0.64938, std: 0.00493, params: {'min_samples_split': 200, 'max_depth': 7},
  mean: 0.64906, std: 0.00490, params: {'min_samples_split': 400, 'max_depth': 7},
  mean: 0.64207, std: 0.00446, params: {'min_samples_split': 600, 'max_depth': 7},
  mean: 0.63941, std: 0.00401, params: {'min_samples_split': 800, 'max_depth': 7},
  mean: 0.63731, std: 0.00397, params: {'min_samples_split': 1000, 'max_depth': 7},
  mean: 0.65832, std: 0.00489, params: {'min_samples_split': 200, 'max_depth': 9},
  mean: 0.65344, std: 0.00592, params: {'min_samples_split': 400, 'max_depth': 9},
  

In [24]:
#let's go with 'min_samples_split' = 200 and max_depth  =13. 
#Also, let's train min_samples leaf and max_features
param_test3 = { 'min_samples_leaf':range(30,71,10),'max_features':range(2,10,2)}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1,max_depth=13, \
                                                               n_estimators=80,min_samples_split=200,\
                                                               subsample=0.8, random_state=10), 
param_grid = param_test3,n_jobs=4,iid=False, cv=5)
gsearch3.fit(train_X,train_y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: 0.62805, std: 0.00365, params: {'max_features': 2, 'min_samples_leaf': 30},
  mean: 0.62422, std: 0.00196, params: {'max_features': 2, 'min_samples_leaf': 40},
  mean: 0.62621, std: 0.00381, params: {'max_features': 2, 'min_samples_leaf': 50},
  mean: 0.62559, std: 0.00505, params: {'max_features': 2, 'min_samples_leaf': 60},
  mean: 0.62559, std: 0.00632, params: {'max_features': 2, 'min_samples_leaf': 70},
  mean: 0.64090, std: 0.00369, params: {'max_features': 4, 'min_samples_leaf': 30},
  mean: 0.64176, std: 0.00361, params: {'max_features': 4, 'min_samples_leaf': 40},
  mean: 0.64125, std: 0.00422, params: {'max_features': 4, 'min_samples_leaf': 50},
  mean: 0.64344, std: 0.00454, params: {'max_features': 4, 'min_samples_leaf': 60},
  mean: 0.64055, std: 0.00429, params: {'max_features': 4, 'min_samples_leaf': 70},
  mean: 0.64617, std: 0.00709, params: {'max_features': 6, 'min_samples_leaf': 30},
  mean: 0.64883, std: 0.00438, params: {'max_features': 6, 'min_samples_leaf

In [25]:
#Let's go with subsample=0.8 for now.
#First tuned  model

gbm_tuned_1 = GradientBoostingClassifier(learning_rate=0.1,max_depth=13, \
                                        n_estimators=80,min_samples_split=200,\
                                         subsample=0.8, random_state=10, max_features=8, min_samples_leaf=40)
gbm_tuned_1.fit(train_X, train_y)
print(gbm_tuned_1.score(valid_X, valid_y))

0.6603125


In [26]:
#Let's tune n_estimators and learning_rate simultaneosuly. decrease by 2 and increase by 2
gbm_tuned_2 = GradientBoostingClassifier(learning_rate=0.05,max_depth=13, \
                                        n_estimators=160,min_samples_split=200,\
                                         subsample=0.8, random_state=10, max_features=8, min_samples_leaf=40)
gbm_tuned_2.fit(train_X, train_y)
print(gbm_tuned_2.score(valid_X, valid_y))

0.66515625


In [30]:
'''
Create Submissions - Best model 
'''

import csv

#make predictions
print('Making predictions')
preds = gbm_tuned_2.predict(X_test)

def create_submission(predictions, test_id):
     with open('submit_gbm.csv', 'wb') as mycsvfile:
            thedatawriter = csv.writer(mycsvfile)
            thedatawriter.writerow(['Id','label'])
            for pred,testid  in zip(predictions,test_id):
                out = [testid,pred+1]
                thedatawriter.writerow(out)
    

print('Creating Submission')
create_submission(preds, X_test_id)

Making predictions
Creating Submission
